In [1]:
## validate model - with step size
# pip install import-ipynb
import import_ipynb
from hdEMG_modules_yw import load_data, build_model, crossValidate, train_model, model_validata, load_data_mat, display_model, evaluate, savedata, get_cnn1d_api, shuffle_data, split_data
import os
from tensorflow.keras.models import load_model
import gc
import time
import numpy as np
from IPython.display import clear_output

def fileFilter(mfilter):
    mfilter = mfilter.replace("]", "[]]")
    mfilter = mfilter.replace("[", "[[]", 1)
    return mfilter

nn_nodes = [128, 128, 128, 64, 256]
global emg_folder_global
emg_folder_global = 'G:\\Across subjects and muscles\\[3] updated training data\\' # EMG folder to the segmented dataset

def train_DNN(trial, mu, train_seg, step_size = 10, window_size = 120, multiFlag = 0):
    # multiFlag = 0 for inidividual motor unit
    # multiFlag = 1 for multiple motor units
    
    modelType = ["nn", "cnn", "rnn"] # 0: nn; 1:cnn; 2:rnn
    modelIndex = 1
    
    # change mu parameters
    if type(mu) is list:
        mucnt = len(mu)
    else:
        mucnt = len([mu])
        
    if mucnt<20:
        mu_prefix = mu
    else:
        mu_prefix = [mu[0], mu[-1]]
    
    # load train data: all three segments
    x_train, y_train = load_data_mat(TR = trial, SG = train_seg, ST = step_size, WS = window_size, MU = mu, TF = 1, MutiSeg = 2, emg_folder_global = emg_folder_global)
    print(x_train.shape)

    # build and train model
    if multiFlag == 1:
        # multiple motor units decomposition
        print('Train for MU {}'.format(mu))
        # construct prefix string
        prefix = "{}-{}-SG{}-ST{}-WS{}-MU{}".format(modelType[modelIndex], trial, train_seg, step_size, window_size, mu_prefix)
        ## build and train model
        model = build_model(modelIndex, WS = window_size, n_output = mucnt, nn_nodes = nn_nodes)
        model.summary()
        model, tname = train_model(model, x_train, y_train, prefix, epochs = 100)
        model_validata(model, x_train, y_train, prefix)
        #     model.summary()
        del x_train, y_train
    else:
        # single motor unit decomposition
        mu_list = mu
        index = 0
        # go through each individual motor unit
        for mu in mu_list:
            print('Train for MU {}'.format(mu))
            # construct prefix string
            prefix = "{}-{}-SG{}-ST{}-WS{}-MU{}".format(modelType[modelIndex], trial, train_seg, step_size, window_size, mu_prefix)
            model = build_model(modelIndex, WS = window_size)
            model, tname = train_model(model, x_train, y_train[index], prefix)
            index = index + 1
    
    del model
    gc.collect()      


def test_DNN(trial, mu_list, test_seg, step_size = 10, window_size = 120, multiFlag = 0, modelpostfix = 'l', test_step_size = None, test_trial = None, test_mu = None):
    # test_trial is the same as model_trial
    # multiFlag = 0 for inidividual motor unit
    # multiFlag = 1 for multiple motor units
    crossFlag = False
    if test_step_size == None:
        test_step_size = step_size

    if test_trial == None:
        test_trial = trial
    else:
        crossFlag = True
        
    if test_mu == None:
        test_mu = mu_list
        
    modelType = ["nn", "cnn", "rnn"]
    modelIndex = 1
    
    train_list = [0, 1, 2]
    
    # load test data
    x_data, y_data = load_data_mat(TR = test_trial, SG = test_seg, ST = test_step_size, WS = window_size, MU = test_mu, emg_folder_global = emg_folder_global)
    
    # return if file does not exist
    if len(x_data) == 0:
        return 0
    
    mu = mu_list
    # change mu parameters
    if type(mu) is list:
        mucnt = len(mu)
    else:
        mucnt = len([mu])
        
    if mucnt<20:
        mu_prefix = test_mu
    else:
        mu_prefix = [test_mu[0], test_mu[-1]]
    
    if multiFlag == 1:
        # evaluate mutiple motor units decomposition
        x_test = x_data
        y_test = y_data
        for train_seg in train_list:
            # get trained model list
            modelFilter = 'best_*cnn-{}-SG{}-ST{}-WS{}-MU{}*_{}.h5'.format(trial, train_seg, step_size, window_size, mu_prefix, modelpostfix)
            modelFilter = fileFilter(modelFilter)
            fileList = glob.glob(modelFilter)
            if len(fileList) == 0:
                print('Cannot find {}'.format(modelFilter))
            # go through each model
            for modelName in fileList: #[0:]:
                print(modelName)
                model = load_model_custom(modelName)
                prefix = "{}-{}-SG{}-ST{}-WS{}-MU{}".format(modelType[modelIndex], test_trial, train_seg, test_step_size, window_size, mu_prefix)

                # extract time label in each modelName
                nameItems = modelName.split('_')
                if len(nameItems[-1])>5:
                    tname = nameItems[-1][0:-3]
                else:
                    tname = "{}_{}".format(nameItems[-2], nameItems[-1][0:-3])
                if crossFlag:
                    prefix4file = "{}-TSG{}-T{}-CV{}".format(prefix, test_seg, tname, trial)
                else:
                    prefix4file = "{}-TSG{}-T{}".format(prefix, test_seg, tname)
                model_validata(model, x_test, y_test, prefix4file)
    else: 
        # evaluate single motor unit decomposition
        index = 0
        for mu in mu_list:   # number of channels
            x_test = x_data
            y_test = y_data[index]
            index = index + 1
            # test with trained model
            for train_seg in train_list:
                # get trained model list
                fileList = glob.glob('best_*cnn-{}-SG{}-ST{}-WS{}-MU{}_*_{}.h5'.format(trial, train_seg, step_size, window_size, mu, modelpostfix))
                # go through each model
                for modelName in fileList[0:]:
                    print(modelName)
                    model = load_model_custom(modelName)
                    prefix = "{}-{}-SG{}-ST{}-WS{}-MU{}".format(modelType[modelIndex], trial, train_seg, step_size, window_size, mu)

                    # extract time label in each modelName
                    nameItems = modelName.split('_')
                    if len(nameItems[-1])>5:
                        tname = nameItems[-1][0:-3]
                    else:
                        tname = "{}_{}".format(nameItems[-2], nameItems[-1][0:-3])

                    prefix4file = "{}-TSG{}-T{}".format(prefix, test_seg, tname)
                    model_validata(model, x_test, y_test, prefix4file)
    gc.collect()

importing Jupyter notebook from hdEMG_modules_yw.ipynb


In [2]:
import glob

def train_test_DNN(trial, mu_list, test_trials=None, train=True, step_list = [5, 10, 20, 40], window_list = [20, 40, 80, 120]):
    
    if test_trials is None:
        test_trials = [trial]
        
    # initialize parameters
    window_size = 120
    step_size = 20
    train_seg = 0
    test_seg = 0
    
    multiFlag = 1
    
    # set testing conditions
    train_list = [0, 1, 2]
    test_list = [0, 1, 2]
    
    ## training the deep cnn
    # loop through step size list and window size list
    if train:
        for train_seg in train_list:   # train seg
            for step_size in step_list:  # step size
                for window_size in window_list:  # step size
                    prefix = "{}-SG{}-ST{}-WS{}-MU{}".format(trial, train_seg, step_size, window_size, mu_list)
                    prefix = fileFilter(prefix)
                    if not glob.glob('*{}*.h5'.format(prefix)):
                        train_DNN(trial, mu_list, train_seg, step_size = step_size, window_size = window_size, multiFlag = multiFlag)
                        time.sleep(1)
                        gc.collect()
                    else:
                        print(prefix)
                        
    ### testing the deep cnn
    # loop through step size list and window size list
    for test_trial in test_trials:
        print(test_trial)
        for step_size in step_list:  # step size
            for window_size in window_list:  # step size
                for test_seg in test_list:   # segments
                    prefix = "{}-*-ST{}-WS{}-MU{}*-TSG{}*_f*{}".format(test_trial, step_size, window_size, mu_list, test_seg, trial)
                    prefix = fileFilter(prefix)
                    if not glob.glob('*{}*.csv'.format(prefix)):
                        test_DNN(trial, mu_list, test_seg, step_size = step_size,  window_size = window_size, multiFlag = multiFlag, test_trial=test_trial, modelpostfix = 'vf')
                        time.sleep(1)
                        gc.collect()
                    else:
                        print(prefix)

In [ ]:
################## Neural drive estimation - 3-fold - Across muscles and sessions (d1VL, d1VM, d2VL, and d2VM) ##################
# 3-fold cross validation for each participant
# Iteration 1: segment 1 and 2 from d1VM, d2VL, d2VM, d1VL -> segment 3 from d1VM, d2VL, d2VM, d1VL
# ...
# Iteration 3: segment 1 and 3 from d1VM, d2VL, d2VM, d1VL -> segment 2 from d1VM, d2VL, d2VM, d1VL
# file name example: allexM0_c8b_uc4-SG1-WS40-ST20.mat
#     allexM0_c8b_uc4-SG1-WS40-ST20.mat; for participant 1, data from segment 1
#     allexM0_c8b_uc4-SG2-WS40-ST20.mat; for participant 1, data from segment 2
#     allexM0_c8b_uc4-SG3-WS40-ST20.mat; for participant 1, data from segment 3

subjectList = [4, 8, 10, 11, 25, 26]
exMuscleList = [0]
step_size = 20
window_size = 40
mu_list = [0, 1, 2, 3]
train_flag = True
prefix = 'allexM';
prefix_test = 'uc'
for subjectID in subjectList:
    for exMuscle in exMuscleList:
        train_trials = [
                        '{}{}_c8b_uc{}'.format(prefix, exMuscle, subjectID),
        ]    
        test_trials = [
                    '{}{}_d1L_VL'.format(prefix_test, subjectID),
                    '{}{}_d1M_VM'.format(prefix_test, subjectID),
                    '{}{}_d2L_VL'.format(prefix_test, subjectID),
                    '{}{}_d2M_VM'.format(prefix_test, subjectID),
        ]
        for trial in train_trials:
            train_test_DNN(trial, mu_list, test_trials=test_trials, train=train_flag, step_list = [step_size], window_list = [window_size])

In [ ]:
################## Neural drive estimation - LOOCV - Across muscles and sessions (d1VL, d1VM, d2VL, and d2VM) ##################
# 4-fold cross validation (leave one out) for each participant
# Iteration 1: d1VM, d2VL, d2VM -> d1VL 
# ...
# Iteration 4: d1VL, d1VM, d2VL -> d2VM
# file name example: allexM1_c8b_uc4-SG1-WS40-ST20.mat

subjectList = [4, 8, 10, 11, 25, 26]
exMuscleList = [1, 2, 3, 4]
step_size = 20
window_size = 40
mu_list = [0, 1, 2, 3]
train_flag = True
prefix = 'allexM';
prefix_test = 'uc'
for subjectID in subjectList:
    for exMuscle in exMuscleList:
        train_trials = [
                        '{}{}_c8b_uc{}'.format(prefix, exMuscle, subjectID),
        ]    
        test_trials = [
                    '{}{}_d1L_VL'.format(prefix_test, subjectID),
                    '{}{}_d1M_VM'.format(prefix_test, subjectID),
                    '{}{}_d2L_VL'.format(prefix_test, subjectID),
                    '{}{}_d2M_VM'.format(prefix_test, subjectID),
        ]
        for trial in train_trials:
            train_test_DNN(trial, mu_list, test_trials=test_trials, train=train_flag, step_list = [step_size], window_list = [window_size])

In [ ]:
################## Neural drive estimation - 3-fold - Across subject (S1-Sn)##################
# 3-fold cross validation for each muscle
# Iteration 1: segment 1 and 2 from S1, S2, S3, S4, S5 -> segment 3 from S1, S2, S3, S4, S5
# ...
# Iteration 3: segment 1 and 3 from S1, S2, S3, S4, S5 -> segment 2 from S1, S2, S3, S4, S5
# file name example: 
#     allex0_c8b_d1M-SG1-WS40-ST20.mat; for d1M muscle, data from segment 1
#     allex0_c8b_d1M-SG2-WS40-ST20.mat; for d1M muscle, data from segment 2
#     allex0_c8b_d1M-SG3-WS40-ST20.mat; for d1M muscle, data from segment 3

subjectList = [4, 8, 11, 25, 26]
exsubjectList = [0]
step_size = 20
window_size = 40
mu_list = [0, 1, 2, 3]
train_flag = True
prefix = 'allex';
train_postfix = 'd1M'    # change to 'd1L', 'd1M', 'd2L', or 'd2M'
test_postfix = 'd1M_VM'  # change to 'd1L_VL', 'd1M_VM', 'd2L_VL', or 'd2M_VM'
for subjectID in exsubjectList:
    train_trials = [
                    '{}{}_c8b_{}'.format(prefix, subjectID, train_postfix),
    ]    
    test_trials = [
                    'uc4_{}'.format(test_postfix),
                    'uc8_{}'.format(test_postfix),
                    'uc11_{}'.format(test_postfix),
                    'uc25_{}'.format(test_postfix),
                    'uc26_{}'.format(test_postfix),
    ]
    for trial in train_trials:
        train_test_DNN(trial, mu_list, test_trials=test_trials, train=train_flag, step_list = [step_size], window_list = [window_size])

In [ ]:
################## Neural drive estimation - LOOCV - Across subject (S1-Sn)##################
# 5-fold cross validation (leave one out) for each muscle
# Iteration 1: S1,S2,S3,S4 -> S5
# ...
# Iteration 5: S2,S3,S4,S5 -> S1
# file name example: 
#     allex4_c8b_d1M-SG1-WS40-ST20.mat; for d1M muscle and exclude S4
subjectList = [4, 8, 11, 25, 26]
exsubjectList = [1, 2, 3, 4, 5]
step_size = 20
window_size = 40
mu_list = [0, 1, 2, 3]
train_flag = True
prefix = 'allex';
test_postfix = 'd1M_VM'  
train_postfix = 'd1M'
for subjectID in exsubjectList:
    train_trials = [
                    '{}{}_c8b_{}'.format(prefix, subjectID, train_postfix),
    ]    
    test_trials = [
                    'uc4_{}'.format(test_postfix),
                    'uc8_{}'.format(test_postfix),
                    'uc11_{}'.format(test_postfix),
                    'uc25_{}'.format(test_postfix),
                    'uc26_{}'.format(test_postfix),
    ]
    for trial in train_trials:
        train_test_DNN(trial, mu_list, test_trials=test_trials, train=train_flag, step_list = [step_size], window_list = [window_size])

In [ ]:
################## Neural drive estimation - mixed validation ##################
# Mixed validation 
# number of dataset included: 1, 2, 3, 4, 5
# number of runs: 1, 2, 3, 4, 5
# test trials: all 20 trials

step_size = 20
window_size = 40
mu_list = [0, 1, 2, 3]
train_flag = True

# get all 20 test trials
subjectList = [4, 8, 11, 25, 26]
test_postfix_list = ['d1L_VL', 'd1M_VM', 'd2L_VL', 'd2M_VM']
test_trials = []
for subjectID in subjectList:
    for test_postfix in test_postfix_list:
        test_trials.append("uc{}_{}".format(subjectID, test_postfix))


# run 1-5
number_of_datasets = 4
prefix = 'allrun'
if number_of_datasets == 1:
    runList = list(range(1, 21))
else:
    runList = [1, 2, 3, 4, 5]  

for runID in runList:
    train_trials = [
                    '{}{}_c8b_mix{}'.format(prefix, runID, number_of_datasets),
    ]    
    for trial in train_trials:
        train_test_DNN(trial, mu_list, test_trials=test_trials, train=train_flag, step_list = [step_size], window_list = [window_size])
        gc.collect()

In [ ]:
################## Neural drive estimation - Across muscles and sessions (d1VL, d1VM, d2VL, and d2VM) ##################
# between-muscle cross validation
# Run 1: d1VL -> d1VM, d2VL, d2VM
# Run 2: d1VM -> d1VL, d2VL, d2VM
# Run 3: d2VL -> d1VL, d1VM, d2VM
# Run 4: d2VM -> d1VL, d1VM, d2VL
subjectList = [4, 8, 11, 25, 26]
step_size = 20
window_size = 40
mu_list = [0, 1, 2, 3]
train_flag = True
prefix = 'uc';
for subjectID in subjectList:
    train_trials = [
                    '{}{}_d1L_VL'.format(prefix, subjectID),
                    '{}{}_d1M_VM'.format(prefix, subjectID),
                    '{}{}_d2L_VL'.format(prefix, subjectID),
                    '{}{}_d2M_VM'.format(prefix, subjectID),
    ]    
    test_trials = [
                    '{}{}_d1L_VL'.format(prefix, subjectID),
                    '{}{}_d1M_VM'.format(prefix, subjectID),
                    '{}{}_d2L_VL'.format(prefix, subjectID),
                    '{}{}_d2M_VM'.format(prefix, subjectID),
    ]
    for trial in train_trials:
        train_test_DNN(trial, mu_list, test_trials=test_trials, train=train_flag, step_list = [step_size], window_list = [window_size])